In this mini project, we will use `Background Subtraction in Dynamic DataSet` from `Statistical Visual Computing Laboratory (SVCL)` at UCSD to experiment data to remove objects other than the street from several security videos using `Singular Value Decomposition` to help test their vision system.


** This exercise is a replicate version from IBM Unsupervised learning Hands-on project.**

In [34]:
import skillsnetwork
# print(skillsnetwork.read.__doc__)
# print(dir(skillsnetwork))
base_path = "E:\\Python\\2024_machine_learning_projects\\introduction\\SVD"
path="E:\\Python\\2024_machine_learning_projects\\introduction\\SVD\\traffic.tar.gz"
# await skillsnetwork.prepare(f"{path}")
await skillsnetwork.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/peds.tar.gz", path=base_path)
await skillsnetwork.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/boats.tar.gz", path=base_path)


Saved as 'peds.tar.gz'


Saved as 'boats.tar.gz'


In [43]:
import pandas as pd
import numpy as np 


from os import listdir,getcwd
from os.path import isfile, join
from random import randint
from PIL import Image

import seaborn as sns 
import matplotlib.pylab as plt
%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD


from sympy import Matrix, init_printing,Symbol
from numpy.linalg import qr,eig,inv,matrix_rank,inv,svd
init_printing()

In [41]:
import tarfile
import os

def extract_tar_gz(file_path, extract_path='.'):
    """ Extract a .tar.gz file to the specified directory.
     Params:
     file_path (str) : the path to the .tar.gz file.
     extract_path (str): The directory where files should be extracted. Default is the current directory.
    """
    try:
        with tarfile.open(file_path, 'r:gz') as tar:
            tar.extractall(path=extract_path)
        print(f"Extracted {file_path} to {extract_path}")
    except Exception as e:
        print(f"An error occured: {e}")

list_tar_files = [os.path.join(os.getcwd(), f) for f in os.listdir() if "tar" in f.split(".")[1]]

for tar_file in list_tar_files:
    extract_tar_gz(tar_file)



Extracted e:\Python\2024_machine_learning_projects\introduction\SVD\boats.tar.gz to .
Extracted e:\Python\2024_machine_learning_projects\introduction\SVD\peds.tar.gz to .
Extracted e:\Python\2024_machine_learning_projects\introduction\SVD\traffic.tar.gz to .


### Defining a helper function to get the data from the matrix.

In [ ]:
def get_data_Matrix (mypath="peds"):
    cwd = getcwd()

    mypath=join(cwd,mypath)
    files = [ join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f)) and f.startswith(".")==False]
    # Read image
    img = Image.open(files[0])
    I=np.array(img)
    # Output Images

    Length,Width=I.shape
   
    X=np.zeros((len(files),Length*Width))
    for i,file in enumerate(files):
        img = Image.open(file)
        I=np.array(img)
        X[i,:]=I.reshape(1,-1)
    return X,Length,Width

## Singular Value Decomposition
The Singular-Value Decomposition, or SVD for short, will decompose a real or complex $N × D$ matrix $\mathbf{X}$ of rank $r$ as follows:


$$\mathbf{X}= \mathbf {US V^{T}}$$


In many applications  $N \ge D$, but SVD can be used for any matrix $\mathbf{X}$. For example, in computer vision and image processing tasks we sometimes have $D \ge N$.


The matrix $\mathbf{S}$ contains the nonnegative <b>singular values</b> of $\mathbf{X}$, with diagonal entries ${\sigma _{i}}$ else $0$, the entries are ordered by importance in descending order with respect to $i$, i.e:

$$\sigma _{1}>\sigma _{2},..>\sigma _{r}$$


The matrix $\mathbf {U}$ is $NxD$ and  has the orthornormal columns  $\mathbf{u}_{1}, ..., \mathbf{u}_{D} $ called the <b>left singular vectors</b>.

The matrix $\mathbf {V}$ is $DxD$ and has the orthornormal columns $\mathbf{v}_{1}, ..., \mathbf{v}_{D}$ called the <b>right singular vectors</b>  (note  that $\mathbf{V}$ transpose, $\mathbf {V^{T}}$ is returned as output in numpy's `svd` function).


 SVD decomposition returns the full shape of a non-square matrix, the non colored  parts of the decomposition **N-D** terms in the matrix U are zeros, we see many of the squares are redundant.


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/images/matrix.png" width="600" alt="full svd">


Consider the matrix $\mathbf{X}$:


In [44]:
X=np.array([[1.0,2],[2,1],[3,3]])
Matrix(X)

⎡1.0  2.0⎤
⎢        ⎥
⎢2.0  1.0⎥
⎢        ⎥
⎣3.0  3.0⎦